In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !%cd /content/drive/MyDrive/zeroshot_new_paper

In [36]:
# import sys
# sys.path.insert(0,'/content/drive/MyDrive/zeroshot_new_paper')

Found cached dataset ag_news (/Users/alealcoforado/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


# === KeyZeroBERTo ===

# Dependencies

In [37]:
# !pip install transformers
# !pip install keybert==0.5.0
# !pip install datasets
# !pip install flair|
# %env PYTORCH_ENABLE_MPS_FALLBACK=1


In [38]:
# !pip install pandarallel

In [39]:
# !pip install evaluate

In [464]:
%load_ext autoreload
%autoreload 2
import nltk
import ipywidgets
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from transformers import pipeline
import torch
import datetime
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK']="1"
import datasets_handler
import evaluation_metrics
import zeroberto
# def getAgora():
#     return str(datetime.datetime.now().strftime("%Y_%m_%d__%H_%M_%S"))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Data Prep

## Source

In [465]:
# which_dataset = 'folhauol' 
# which_dataset = 'bbc-news'
which_dataset = 'ag_news'

# hyp_template = "{}."
# hyp_template = "O tema principal deste texto é {}."
hyp_template = "this text is about {}."
# hyp_template = "this article is about {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)


Found cached dataset ag_news (/Users/alealcoforado/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

## Dataset build

### Train Data

In [466]:
train = raw_data
len(train),train.columns

(127600, Index(['index', 'text', 'label', 'label_text'], dtype='object'))

In [467]:
labels_list = list(train[class_col].unique())
print(labels_list)

['business', 'science and technology', 'sports', 'world']


# Sentence Extraction

In [468]:
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()

In [469]:
train['sentences'] = train[data_col].progress_apply(lambda txt:tokenizer.tokenize(txt))
train.head(2)   

100%|██████████| 127600/127600 [00:01<00:00, 117392.05it/s]


,index,text,label,label_text,sentences
0,0,Wall St. Bears Claw Back Into the Black (Reute...,2,business,"[Wall St., Bears Claw Back Into the Black (Reu..."
1,1,Carlyle Looks Toward Commercial Aerospace (Reu...,2,business,[Carlyle Looks Toward Commercial Aerospace (Re...


In [470]:
sentences_to_classify = []
for sentences in train['sentences']:
  try:
    for sentence in sentences:
      sentences_to_classify.append(sentence)
      break
  except:
    print(train['sentences'])
train['sentence_to_zeroshot'] = sentences_to_classify
len(train)


127600

In [471]:
train['len_sentences'] = train['sentences'].progress_apply(lambda txt:len(txt))
# train[['len_sentences']].describe()

100%|██████████| 127600/127600 [00:00<00:00, 1741553.07it/s]


# Zero-Shot

## Parameters

In [478]:
# max_inferences = len(train) #estava em 5000
max_inferences = 3000

# zeroshot_method = "probability_threshold"
probability_goal = 0.9

zeroshot_method = "top_n_goal"  
top_n_goal = 8

zeroshot_config = {
    'dataset':which_dataset,
    'method':zeroshot_method,
    'prob_goal':probability_goal,
    'top_n_goal':top_n_goal,
    'max_inferences':max_inferences,
    'labels':labels_list,#list(dict_classes_folha.values())
    'template': hyp_template,
    'random_state':422
}
zeroshot_config

{'dataset': 'ag_news',
 'method': 'top_n_goal',
 'prob_goal': 0.9,
 'top_n_goal': 8,
 'max_inferences': 3000,
 'labels': ['business', 'science and technology', 'sports', 'world'],
 'template': 'this text is about {}.',
 'random_state': 422}

## Data

In [479]:
zeroshot_data = train.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state'])
len(zeroshot_data)

3000

## Model

In [480]:
classifier = pipeline("zero-shot-classification",
                      model="joeddav/xlm-roberta-large-xnli",device="cpu") # device = 'cuda'  ##X## DESCOMENTAR AQUI 

# classifier = zeroberto.ZeroBERTo(classes_list=zeroshot_config['labels'],hypothesis_template=zeroshot_config['template'])  ##X## COMENTAR AQUI 

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Inference

In [481]:
%%time
zeroshot_results = zeroberto.runZeroShotPipeline(classifier,zeroshot_data['sentence_to_zeroshot'],zeroshot_config)  ##X## DESCOMENTAR AQUI 

# zeroshot_results = zeroberto.runZeroberto(classifier,zeroshot_data['sentence_to_zeroshot'],zeroshot_config)  ##X## COMENTAR AQUI 
# print(len(zeroshot_results))
### 100 predições ---> <240 segundos (4 min)
### 200 predições ---> <480 segundos (8 min)
### 1000 predições ---> <2400 segundos (40 min)
### 5000 predições ---> 12000 segundos (200 min) (3h 20min)

# data: 3000
{'dataset': 'ag_news', 'method': 'top_n_goal', 'prob_goal': 0.9, 'top_n_goal': 8, 'max_inferences': 3000, 'labels': ['business', 'science and technology', 'sports', 'world'], 'template': 'this text is about {}.', 'random_state': 422}
Preds: 50  - Total time: 34.22 seconds - ETA: 34.2 minutes
Preds: 100  - Total time: 63.26 seconds - ETA: 31.6 minutes
Preds: 150  - Total time: 92.42 seconds - ETA: 30.8 minutes
Preds: 200  - Total time: 121.87 seconds - ETA: 30.5 minutes
Preds: 250  - Total time: 150.48 seconds - ETA: 30.1 minutes
Preds: 300  - Total time: 179.89 seconds - ETA: 30.0 minutes
Preds: 350  - Total time: 208.42 seconds - ETA: 29.8 minutes
Preds: 400  - Total time: 237.26 seconds - ETA: 29.7 minutes
Preds: 450  - Total time: 269.28 seconds - ETA: 29.9 minutes
Preds: 500  - Total time: 297.38 seconds - ETA: 29.7 minutes
Preds: 550  - Total time: 325.71 seconds - ETA: 29.6 minutes
Preds: 600  - Total time: 355.38 seconds - ETA: 29.6 minutes
Preds: 650  - Total time:

In [482]:
label_probabilities_df = pd.DataFrame(zeroshot_results,columns=zeroshot_config['labels'])  ##X## COMENTAR AQUI 

In [483]:
# label_probabilities_df = zeroberto.formatZeroShotResults(zeroshot_results) ##X## DESCOMENTAR AQUI 

In [484]:
def getTopLabel(row):
  return row.idxmax()
def getTopProb(row):
  return row.max()

label_results = label_probabilities_df.apply(getTopLabel,axis=1)
prob_results = label_probabilities_df.apply(getTopProb,axis=1)
label_results_df = pd.Series(label_results,name='prediction')


In [485]:
true_labels_df = raw_data[raw_data.index.isin(train.index)][class_col]
final_result_df = pd.concat([true_labels_df,label_results_df],axis=1)

In [489]:
vector_true = ( final_result_df[class_col].astype(str).str.strip().str.lower() 
               == final_result_df['prediction'].astype(str).str.strip().str.lower() )

## Metrics

In [490]:
final_result_df_encoded = evaluation_metrics.Encoder(final_result_df,['prediction',class_col])

In [491]:
try:print(all_metrics)
except:pass
all_metrics = evaluation_metrics.get_metrics(final_result_df_encoded['prediction_code'][0:len(zeroshot_results)].to_list(),
                               final_result_df_encoded[class_col+'_code'][0:len(zeroshot_results)].to_list())
print(all_metrics)

{'weighted': [{'accuracy': 0.0516}, {'precision': 0.28640307785483954}, {'recall': 0.0516}, {'f1': 0.05945623882399389}], 'macro': [{'accuracy': 0.0516}, {'precision': 0.12025449358935446}, {'recall': 0.09231527022808005}, {'f1': 0.02599786802333615}]}


/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alealcoforado/opt/anaconda3/envs/venv_zeroberto/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'weighted': [{'accuracy': 0.23466666666666666}, {'precision': 0.05506844444444444}, {'recall': 0.23466666666666666}, {'f1': 0.08920374370050395}], 'macro': [{'accuracy': 0.23466666666666666}, {'precision': 0.058666666666666666}, {'recall': 0.25}, {'f1': 0.09503239740820735}]}


### Probabilities

In [ ]:
df_predictions_probs = pd.concat([final_result_df_encoded,
                                  pd.Series(prob_results,name='top_probability'),
                                  pd.Series(vector_true,name='right_wrong')],axis=1)

##### Top N results


In [ ]:
def get_top_n_results(dataframe_results,top_n=1):
    df_top_n = dataframe_results.sort_values(['top_probability','prediction'], ascending=False).groupby('prediction').head(top_n)

    accuracy_top_n = df_top_n.right_wrong.sum()/len(df_top_n.right_wrong)

    accuracy = { "accuracy in top {}".format(top_n) : round(accuracy_top_n,3)}
    print(accuracy)
    return 

get_top_n_results(df_predictions_probs,top_n = 1)
get_top_n_results(df_predictions_probs,top_n = 2)
get_top_n_results(df_predictions_probs,top_n = 3)
get_top_n_results(df_predictions_probs,top_n = 4)
get_top_n_results(df_predictions_probs,top_n = 5)
get_top_n_results(df_predictions_probs,top_n = 6)
get_top_n_results(df_predictions_probs,top_n = 7)
get_top_n_results(df_predictions_probs,top_n = 8)
get_top_n_results(df_predictions_probs,top_n = 9)
get_top_n_results(df_predictions_probs,top_n = 10)
get_top_n_results(df_predictions_probs,top_n = 11)
get_top_n_results(df_predictions_probs,top_n = 12)
get_top_n_results(df_predictions_probs,top_n = 13)
get_top_n_results(df_predictions_probs,top_n = 14)
get_top_n_results(df_predictions_probs,top_n = 15)
get_top_n_results(df_predictions_probs,top_n = 16)
get_top_n_results(df_predictions_probs,top_n = len(zeroshot_results))

{'accuracy in top 1': 0.0}
{'accuracy in top 2': 0.048}
{'accuracy in top 3': 0.1}
{'accuracy in top 4': 0.079}
{'accuracy in top 5': 0.065}
{'accuracy in top 6': 0.075}
{'accuracy in top 7': 0.1}
{'accuracy in top 8': 0.106}
{'accuracy in top 9': 0.111}
{'accuracy in top 10': 0.117}
{'accuracy in top 11': 0.11}
{'accuracy in top 12': 0.103}
{'accuracy in top 13': 0.12}
{'accuracy in top 14': 0.124}
{'accuracy in top 15': 0.127}
{'accuracy in top 16': 0.121}
{'accuracy in top 5000': 0.052}


## Save Results

In [437]:
evaluation_metrics.saveZeroshotResults(zeroshot_config,df_predictions_probs)

predictions_and_probabilities_test_2023_02_13__19_40_16.csv
zeroshot_config_test_2023_02_13__19_40_16.csv


'2023_02_13__19_40_16'

# Load Previous Results

In [30]:
# df_test_results = pd.read_csv(r'/Users/alealcoforado/Documents/Projetos/Datasets/predictions_and_probabilities_test1.csv')

In [31]:
# df_test_results.index = df_test_results['Unnamed: 0']
# df_test_results.drop(columns="Unnamed: 0",inplace=True)